In [1]:
import os
from glob import glob
import pandas as pd
import html
import itertools
import pickle
import re
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.naive_bayes import MultinomialNB,GaussianNB
from sklearn.cluster import KMeans, k_means_, DBSCAN, AgglomerativeClustering
from sklearn import cluster
from sklearn.metrics import silhouette_score
from sklearn.metrics.pairwise import cosine_distances

import hdbscan
from gensim import models
from gensim.corpora import mmcorpus, Dictionary
from gensim.models import lsimodel, ldamodel, tfidfmodel, rpmodel, logentropy_model, TfidfModel, LsiModel, LdaModel
from gensim import matutils, corpora
from gensim.models.doc2vec import TaggedDocument
from gensim.models import doc2vec
from gensim.models.word2vec import Word2Vec
from sklearn.svm import SVC

import sys
sys.path.append('~/Documents/GitHub/Private_Project/personal_project/')
import html
from collections import Counter
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import chat_bot as cb
import Database_Handler as dh
from datetime import datetime
import seaborn as sns
import functools
from konlpy.tag import Mecab
from konlpy.utils import pprint
mecab = Mecab()
%matplotlib inline

In [2]:
dataDict = pickle.load(open('./data/pre_data/for_statistics_daum_from_mongodb.pickled','rb'))
print (len(dataDict))

4722


In [3]:
keywordsDict = pickle.load(open('./data/pre_data/keywords_daum.pickled','rb'))

In [4]:
for idx in dataDict:
    dataDict[idx]['extracted_keywords'] = keywordsDict[idx]

In [5]:
df_daum = pd.DataFrame.from_dict(dataDict, orient='index')
df_daum['date'] = pd.to_datetime(df_daum['date']).dt.date
df_daum.reset_index(inplace = True)
df_daum.rename(columns={'index':'id'}, inplace=True)
print (df_daum.shape)
df_daum.head()

(4722, 11)


,id,category,date,press,number_of_comment,number_of_crawled_comment,rank,title,mainText,keywords,extracted_keywords
0,5a2a61bf588c13481c229d1e,뉴스,2017-12-07,세계일보,1093,911,1,"""밤이 무섭다""..비아그라 공장 연기에 남성들 부작용 호소","주민들은 공장에서 배출된 연기가 '남성이 매우 건강해지는 부작용'을 일으킨다며, ...","[부작용, 비아그라, 아일랜드]","{지역, 남성들, 부작용, 건강, 공장, 연기, 세보 효과}"
1,5a2a61bf588c13481c229d1f,뉴스,2017-12-07,헬스조선,603,386,2,식후 커피·늦은 양치질..점심식사 후 하면 안 좋은 습관 3가지,점심식사를 마친 후 후식으로 커피를 마시는 사람들이 많다. 실제로 직장이 밀집돼 ...,"[커피, 낮잠, 음식물]","{디스크, 낮잠, 치아, 커피, 식후, 철분, 건강, 입냄새, 자세, 점심 식사}"
2,5a2a61bf588c13481c229d20,뉴스,2017-12-07,연합뉴스,1067,811,3,"'십년지기 생매장' 진짜 이유는..""'청부 통정' 알려질까 봐""",(성남=연합뉴스) 최해민 기자 = 십년지기 지인을 산 채로 묻어 살해한 50대 여...,"[살인혐의, 철원, 검찰송치]","{철원, 지인, 범행, 앙심, 아들, 남편, 진술, 주변, 성관계, 경찰}"
3,5a2a61bf588c13481c229d21,뉴스,2017-12-07,헤럴드경제,418,369,4,"신영자, 억 소리나는 갑질","신영자, 적용안된 혐의→검찰 상고에서 인정\n신영자, 얼마를 어떻게 받았나 [헤럴...","[신영자, 갑질, 롯데백화점]","{매장, 혐의, 유통업체, 징역, 네이처리퍼블릭, 신영자 이사장, 롯데, 검찰}"
4,5a2a61bf588c13481c229d22,뉴스,2017-12-07,연합뉴스,434,368,5,"""배신하지마"" 20대女 살인 피의자 유치장서 공범 남친에 쪽지",(청주=연합뉴스) 이승민 기자 = 지난 9월 청주의 한 하천에서 20대 여성을 둔기...,"[공범, 살인, 과자]","{유치장, 남자친구, 범행, 혐의, 폭행, 과자, 쪽지, 경찰}"


In [6]:
textInfo = df_daum.title.values + df_daum.mainText.values

In [7]:
def nav_tokenizer(corpus):
    pos = mecab.pos(corpus)
    res = [x[0] for x in pos if (x[1] == u'VV' or x[1] == u'VA' or x[1] == u'NNB' or x[1] == u'NNP' or x[1] == u'NNG')]
    return res

In [10]:
def remove_stopwords(words, stopwords):
    res = [x for x in words if x not in stopwords]
    res = [x for x in words if len(x) != 1]
    return res

In [11]:
stopwords = open('./data/stopwordsList.txt',encoding='utf-8').readlines()
stopwords = list(map(lambda x: x.strip(), stopwords))

### doc2vec

In [12]:
d2v_docs = [nav_tokenizer(x) for x in textInfo]
d2v_docs = [remove_stopwords(x, (stopwords)) for x in d2v_docs]

In [85]:
tagged_d2v_docs = [TaggedDocument(doc, [idx]) for idx, doc in enumerate(d2v_docs)]

In [ ]:
d2v_model = doc2vec.Doc2Vec(size=1000, window=8, alpha=0.025, min_alpha=0.025, seed=0)
d2v_model.build_vocab(tagged_d2v_docs)
d2v_model.train_lbls = False # do not train labels of words
d2v_model.train_words = True # only train relations among words

In [ ]:
for epoch in range(30):
    d2v_model.train(tagged_d2v_docs, total_examples=len(d2v_docs), epochs=epoch)
    d2v_model.alpha -= 0.002 # decrease the learning rate
    d2v_model.min_alpha = d2v_model.alpha # fix the learning rate, no decay

In [ ]:
d2v_model.save('./data/pre_data/pre_data_classification_size1500_epoch30_doc2vec_model_by_mecab.d2v')

In [ ]:
X_d2v = [d2v_model.infer_vector(x.words) for x in tagged_d2v_docs]

### word2vec

In [ ]:
w2v_model = Word2Vec(d2v_docs, sg=1, size=3000,window=8)
w2v_model.init_sims(replace = True)

In [ ]:
print(w2v_model.most_similar('신혜선'))

In [ ]:
print(w2v_model.most_similar('손흥민'))

In [ ]:
print(w2v_model.most_similar('황금빛내인생'))

In [ ]:
print(w2v_model.most_similar('홍준표'))

In [ ]:
print(w2v_model.most_similar('문재인'))

In [ ]:
print(w2v_model.most_similar('노무현'))

In [ ]:
print(w2v_model.most_similar('박근혜'))

In [ ]:
print(w2v_model.most_similar('국정원'))

In [ ]:
print(w2v_model.most_similar('이낙연'))

In [ ]:
print(w2v_model.most_similar('이국종'))

In [ ]:
print(w2v_model.most_similar('김성태'))

In [ ]:
print(w2v_model.most_similar('더불어민주당'))

In [ ]:
print(w2v_model.most_similar('새누리당'))

In [ ]:
print(w2v_model.most_similar('자유한국당'))

In [ ]:
print(w2v_model.most_similar('국민의당'))

In [ ]:
print(w2v_model.most_similar('안철수'))

In [ ]:
print(w2v_model.most_similar('유승민'))

In [99]:
w2v_model.save('./data/pre_data/word2vec_category_size1500_by_mecab.w2v')